# Cargamos Librerías

In [1]:
import pandas as pd
import openpyxl
import warnings
import os
import matplotlib.pyplot as plt 
import seaborn as sns
import utils as eda

def warn(*args, **kwargs):
    pass
warnings.warn = warn
warnings.filterwarnings("ignore", category=FutureWarning)

# Cargamos los Dataset

In [2]:
movies = pd.read_csv("../data/interim/movies-data/ml-32m/movies.csv")
tags = pd.read_csv("../data/interim/movies-data/ml-32m/tags.csv")
ratings = pd.read_csv("../data/interim/movies-data/ml-32m/ratings.csv")

# Análisis Exploratorio de Datos

In [3]:
eda.analisis(movies)

Variables numéricas: ['movieId']
Variables no numéricas: ['title', 'genres']

Shape del dataset: (87585, 3)

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  87585 non-null  int64 
 1   title    87585 non-null  object
 2   genres   87585 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB
None

Valores nulos por columna:
movieId    0
title      0
genres     0
dtype: int64

Filas duplicadas:
Empty DataFrame
Columns: [movieId, title, genres]
Index: []


El dataset de películas, *movies*, contiene 3 variables: una variable numérica y dos variables categóricas. Hay 87585  registros en total, sin valores nulos.

In [4]:
eda.analisis(tags)

Variables numéricas: ['userId', 'movieId', 'timestamp']
Variables no numéricas: ['tag']

Shape del dataset: (2000072, 4)

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000072 entries, 0 to 2000071
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   userId     int64 
 1   movieId    int64 
 2   tag        object
 3   timestamp  int64 
dtypes: int64(3), object(1)
memory usage: 61.0+ MB
None

Valores nulos por columna:
tag          17
userId        0
movieId       0
timestamp     0
dtype: int64

Filas duplicadas:
Empty DataFrame
Columns: [userId, movieId, tag, timestamp]
Index: []


El dataset de etiquetas, *tags*, contiene 4 variables: tres variables numéricas y una variable categórica. Hay 2000071 registros en total y la columna *tag* tiene 17 valores nulos.

In [7]:
target='rating'
eda.analisis(ratings) #Primer contacto cn dataset ratings

Variables numéricas: ['userId', 'movieId', 'rating', 'timestamp']
Variables no numéricas: []

Shape del dataset: (32000204, 4)

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000204 entries, 0 to 32000203
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 976.6 MB
None

Valores nulos por columna:
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Filas duplicadas:
Empty DataFrame
Columns: [userId, movieId, rating, timestamp]
Index: []


## Eliminar Duplicados

Verificamos, para cada dataset, si existen duplicados.

In [8]:
movies.drop_duplicates()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
87580,292731,The Monroy Affaire (2022),Drama
87581,292737,Shelter in Solitude (2023),Comedy|Drama
87582,292753,Orca (2023),Drama
87583,292755,The Angry Breed (1968),Drama


No hay filas duplicadas en el dataset *movies*.

Eliminar las filas duplicadas del dataset *tags* requiere otras consideraciones, pues al tratarse de un dataset creado por los usuarios que han visto la película, pueden existir duplicados. Esto lo vemos, por ejemplo, en la película Pulp Fiction, que tiene 6697 etiquetas. Ya que las etiquetas están relacionadas con el momento de la película (*timestamp*), es posible tener la misma etiqueta varias veces.

In [9]:
# Agrupar por movieId y contar tags
tags_count = tags.groupby("movieId")["tag"].count().reset_index()

# Obtener el movieId con más tags
top_movie_id = tags_count.sort_values(by="tag", ascending=False).iloc[0]["movieId"]

print(movies[movies["movieId"] == top_movie_id])

print(tags[tags["movieId"] == top_movie_id].count())

print(tags[(tags["movieId"] == top_movie_id) & (tags["tag"] == "action")])

     movieId                title                       genres
292      296  Pulp Fiction (1994)  Comedy|Crime|Drama|Thriller
userId       6697
movieId      6697
tag          6697
timestamp    6697
dtype: int64
         userId  movieId     tag   timestamp
912         303      296  action  1551552319
1200        332      296  action  1438215918
14228      2372      296  action  1582233289
26160      4859      296  action  1425845174
29995      5143      296  action  1565314945
...         ...      ...     ...         ...
1824963  145435      296  action  1595104559
1867320  147969      296  action  1310019101
1916541  154003      296  action  1275699249
1955157  158906      296  action  1594135958
1991874  160642      296  action  1241009210

[82 rows x 4 columns]


En este caso, no eliminaremos los duplicados en esta instancia, sino que transformaremos los datos para que cada *tag* esté asociado a una película como máximo una sola vez.

## Crear Lista de Géneros

Para poder hacer un mejor análisis de los datos, separaremos los géneros en una lista y transformaremos el registro, para que cada movieId tenga un género por fila.

In [10]:
movies["genres"] = movies["genres"].str.split("|")

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


Crearemos un nuevo dataset en el que cada género esté en una fila diferente. Esto nos permitirá hacer análisis univariantes más adelante.

In [9]:
movie_genre = movies.explode("genres").reset_index(drop=True)

## Un *tag* por película

Ya que parte del análisis requiere ver los datos de forma global, crearemos un nuevo dataset en el los *tags* no estarán agrupados y cada fila represente un *tag* por película.

In [12]:
movie_tag = tags[['movieId', 'tag']].copy()

print(movie_tag.count())

movie_tag.head()

movieId    2000072
tag        2000055
dtype: int64


,movieId,tag
0,26479,Kevin Kline
1,79592,misogyny
2,247150,acrophobia
3,2174,music
4,2174,weird


In [13]:
movie_tag = movie_tag.drop_duplicates()

print(movie_tag.count())

movieId    1086841
tag        1086825
dtype: int64


## Conversión de Fecha en Ratings

In [14]:
# Convertir timestamp a fecha legible
ratings['datetime'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings['year'] = ratings['datetime'].dt.year

print("RANGO TEMPORAL:")
print(f"Fecha más antigua: {ratings['datetime'].min()}")
print(f"Fecha más reciente: {ratings['datetime'].max()}")
print(f"Años cubiertos: {ratings['year'].min()} - {ratings['year'].max()}")
print("\n" + "="*50)
ratings.head()

RANGO TEMPORAL:
Fecha más antigua: 1995-01-09 11:46:44
Fecha más reciente: 2023-10-13 02:29:07
Años cubiertos: 1995 - 2023



,userId,movieId,rating,timestamp,datetime,year
0,1,17,4.0,944249077,1999-12-03 19:24:37,1999
1,1,25,1.0,944250228,1999-12-03 19:43:48,1999
2,1,29,2.0,943230976,1999-11-22 00:36:16,1999
3,1,30,5.0,944249077,1999-12-03 19:24:37,1999
4,1,32,5.0,943228858,1999-11-22 00:00:58,1999


In [15]:
# elimina la columna timestamp
ratings.drop(columns=['timestamp'], inplace=True)

# Verificar
print(ratings.head())

   userId  movieId  rating            datetime  year
0       1       17     4.0 1999-12-03 19:24:37  1999
1       1       25     1.0 1999-12-03 19:43:48  1999
2       1       29     2.0 1999-11-22 00:36:16  1999
3       1       30     5.0 1999-12-03 19:24:37  1999
4       1       32     5.0 1999-11-22 00:00:58  1999


In [16]:
movie_rating = (
    ratings.groupby("movieId")["rating"]
    .agg(["mean", "count"])
    .reset_index()
    .rename(columns={"mean": "avg_rating", "count": "num_ratings"})
)

movie_rating.head()


,movieId,avg_rating,num_ratings
0,1,3.897438,68997
1,2,3.275758,28904
2,3,3.139447,13134
3,4,2.845331,2806
4,5,3.059602,13154


In [17]:
# Análisis de valores únicos
print("VALORES ÚNICOS:")
print(f"Usuarios únicos: {ratings['userId'].nunique():,}")
print(f"Películas únicas: {ratings['movieId'].nunique():,}")

VALORES ÚNICOS:
Usuarios únicos: 200,948
Películas únicas: 84,432


In [18]:
# Distribución de ratings por usuario
user_rating_counts = ratings['userId'].value_counts()
print(" DISTRIBUCIÓN DE ACTIVIDAD DE USUARIOS:")
print(f"Usuario más activo: {user_rating_counts.max()} ratings")
print(f"Usuario menos activo: {user_rating_counts.min()} ratings")
print(f"Mediana de ratings por usuario: {user_rating_counts.median()}")

 DISTRIBUCIÓN DE ACTIVIDAD DE USUARIOS:
Usuario más activo: 33332 ratings
Usuario menos activo: 20 ratings
Mediana de ratings por usuario: 73.0


In [19]:
# Distribución de ratings por película
movie_rating_counts = ratings['movieId'].value_counts()
print(" DISTRIBUCIÓN DE POPULARIDAD DE PELÍCULAS:")
print(f"Película más valorada: {movie_rating_counts.max()} ratings")
print(f"Película menos valorada: {movie_rating_counts.min()} ratings")
print(f"Mediana de ratings por película: {movie_rating_counts.median()}")

 DISTRIBUCIÓN DE POPULARIDAD DE PELÍCULAS:
Película más valorada: 102929 ratings
Película menos valorada: 1 ratings
Mediana de ratings por película: 5.0
